In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)

In [2]:
from src.mm_v2.interface import mmkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/mm_v2/build.ninja...
Building extension module mm_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernels.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=mm_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D_

Loading extension module mm_v2...


In [3]:
div = 1
M = 8192 // div
N = 8192 // div
K = 8192 // div
device = torch.device("cuda:0")
dtype = torch.float32
dtype_AB = torch.bfloat16
beta = 1.2
alpha = 1.1

In [4]:
torch.manual_seed(0)
mat_A = torch.randn((M, K)).to(device=device, dtype=dtype_AB)
mat_B = torch.randn((K, N)).to(device=device, dtype=dtype_AB)
mat_C = torch.zeros((M, N)).to(device=device, dtype=dtype)

In [5]:
mat_D_pt = (beta / alpha) * mat_A @ mat_B + mat_C * alpha
# mat_D2_pt = mat_A @ mat_B.transpose(0,1) + mat_C

In [6]:
mat_D_cu = mmkernel(mat_A, mat_B.transpose(0,1), mat_C)

before kernel dispatch - bfloat16!
Required shared memory size: 73728 bytes = MAX(73728, 65536)
Required shared memory size: 72 Kb
#bytes: sizeof(int4)= 16, sizeof(float4)= 16, sizeof(__nv_bfloat16)= 2
CHUNK_K=8, CHUNK_LINE_BYTES=256, WARP_COPY_BYTES=512, CHUNK_COPY_LINES_PER_WARP=2, CHUNK_COPY_LINE_LANES=16
BLOCK_ROW_TILES=8, BLOCK_COL_TILES=8, BLOCK_ROW_WARPS=2, BLOCK_COL_WARPS=4
WARP_ROW_TILES=4, WARP_COL_TILES=2, WARP_SIZE=32, WARPS_PER_BLOCK=8
SHMEM_STRIDE=128, SHMEM_OFFSET=64
BlockIdx: 0, ThreadIdx: 0, blockDim: 256, gridDim: 48
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_pos: 0, block_tile_i: 0, block_tile_j: 0
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_pos: 48, block_tile_i: 0, block_tile_j: 384
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_pos: 96, block_tile_i: 8, block_tile_j: 256
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_pos: 144, block_tile_i: 16, block_tile_j: 128
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_pos: 192, block_tile_i: 24, block_tile_j: 0
BIdx: 0, TIdx: 0, wID: 0, lID: 0, block_

In [7]:
mat_D_cu

tensor([[   0.6027,   99.5545,    6.6524,  ...,   63.4920,   90.0374,  -79.6230],
        [ -16.2407,  -70.2106,   74.2199,  ...,  108.1273, -200.2493,   93.9348],
        [ -61.3344,   -7.3290, -164.8177,  ...,    3.9053,  -57.0123, -187.6240],
        ...,
        [  15.8896,   -9.2444,  164.1558,  ...,   21.6965,  -97.8878,  -23.0336],
        [ 140.6422,   72.7060,   11.8973,  ..., -195.9341, -194.0892,  143.7378],
        [ -21.1582, -170.4006,  -10.7048,  ...,  -35.7926,  104.6309, -104.6804]], device='cuda:0')

In [8]:
mat_D_pt

tensor([[   0.6367,   99.0000,    6.7500,  ...,   63.0000,   89.5000,  -79.0000],
        [ -16.0000,  -69.5000,   73.5000,  ...,  107.0000, -199.0000,   93.0000],
        [ -60.7500,   -7.0938, -163.0000,  ...,    4.0312,  -56.5000, -187.0000],
        ...,
        [  15.6875,   -9.4375,  163.0000,  ...,   21.5000,  -97.0000,  -22.8750],
        [ 139.0000,   72.0000,   11.6875,  ..., -195.0000, -192.0000,  143.0000],
        [ -21.0000, -169.0000,  -10.8750,  ...,  -35.5000,  104.0000, -103.5000]], device='cuda:0')